<a href="https://colab.research.google.com/github/basselkassem/easy21/blob/master/policy_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libs

In [1]:
!pip install import-ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=f1fec566ffd7c3017041260ba03bb07513bc03e01255885d760431102798bc13
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [5]:
from google.colab import drive
drive.mount("/content/drive", )
%cd /content/drive/'My Drive'/'Colab Notebooks'/easy21

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks/easy21


# Import libs

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import import_ipynb
import environment as env
from collections import defaultdict
import pickle

# Load optimal Q from MonteCarlo

In [0]:
Q_star = None
with open('q_star.pickle_5', 'rb') as file:
    Q_star = pickle.load(file)

# Implementation of Policy Gradient

In [0]:
class PolicyGradient:
    def __init__(self,):
        pass
    
    def epsilon_greedy(self, state):
        pass
    def agent_policy(self, state):
        pass

    def policy_improvment(self,):
        pass
    def policy_evaluation(self):
        pass
    def policy_iteration(self):
        pass

# Visualize Results